In [ ]:
!pip install langchain_openai python-dotenv streamlit langchain_community langserve fastapi uvicorn sse_starlette bs4 chromadb faiss-cpu gradio pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.8 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-70.1.0-py3-none-any.whl (882 kB)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 1.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 31.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 27.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.6 MB/s eta 0:00:00
    Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-cjggq8fw/overlay/local/bin
    changing mode of /tmp/pip-build-env-cjggq8fw/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-cjggq8fw/overlay/local/bin/wheel to 755
    changing mode of /tmp/pip

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from openai import OpenAI

import gradio as gr
import random
import time
import re

from google.colab import userdata

In [ ]:
# client = OpenAI(api_key=userdata.get('OPENAPIKEY'))


Initialize LLM

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
lcpp_llm.params.n_gpu_layers

32

Calling LLM

In [ ]:
def generate (query):
  prompt = query
  # SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully. Use the context given in the prompt.
  prompt_template = f'''

  USER: {prompt}

  ASSISTANT:
  '''
  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                    repeat_penalty=1.2, top_k=150,
                    echo=True)
  return response["choices"][0]["text"]

In [ ]:
# print (response)
# print(response["choices"][0]["text"])

{'id': 'cmpl-a600fd4d-4e17-4711-b9f6-52ae6e7a1ccf', 'object': 'text_completion', 'created': 1718810318, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully\n\nUSER: Write linear regression in python\n\nASSISTANT:\n\nTo write a linear regression model in Python, you can use scikit-learn library. Here is an example of how to do it:\n```\nfrom sklearn.linear_model import LinearRegression\nimport pandas as pd\n\n# Load the data\ndf = pd.read_csv('data.csv')\n\n# Create a linear regression model\nreg = LinearRegression()\n\n# Train the model using the training data\nreg.fit(df[['x1', 'x2']], df['y'])\n\n# Make predictions on new data\npredictions = reg.predict(df[['x1', 'x2']])\n```\nThis code assumes that you have a CSV file named `data.csv` containing the input features 

Code for Web link

In [ ]:
def web_link(link=None,query=None):

  if(link):
    try:
      loader = WebBaseLoader(web_path=(link,), bs_kwargs=dict(parse_only = bs4.SoupStrainer(
        # class_ = ("post-header", "post-content")
        class_ = ("mw-content-ltr mw-parser-output")
      )))

      webpagedoc = loader.load()

      text_splitter = RecursiveCharacterTextSplitter(chunk_size = 150, chunk_overlap = 50)

      global documents
      documents = text_splitter.split_documents(webpagedoc)

      #vector embedding and storage
      global db
      db = "DB"

      db = Chroma.from_documents(documents, OpenAIEmbeddings(openai_api_key = userdata.get('OPENAPIKEY')))

      return "Link Read"

    except Exception as e:
      return str(e)

  else:
      result = "No Data"
      response = ""
      if(result=="No Data"):
        result = db.similarity_search(query)
        response = result[0].page_content
        first = response
        for i in range(len(result)):
          if first != result[i].page_content:
            response += "\n" + result[i].page_content
            break


        # query = query + response

        # stream = client.chat.completions.create(
        #   model="gpt-3.5-turbo",
        #   messages=[{"role": "user", "content": query}],
        #   stream=True,
        # )
      # for chunk in stream:
      #   if chunk.choices[0].delta.content is not None:
      #       response = response + chunk.choices[0].delta.content
      # pass

      return response

Code for PDF file

In [ ]:
def pdf_file(path, message):
  #implement pdf query logic here

  loader = PyPDFLoader(path)
  pdfdocs = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
  pdfdoc = text_splitter.split_documents(pdfdocs)

  db = Chroma.from_documents(pdfdoc[:50], OpenAIEmbeddings(openai_api_key = userdata.get('OPENAPIKEY')))

  query = message

  result = db.similarity_search(query)

  response = result[0].page_content

  query = query + "\nContext:" + response

  #   # Retrieve the top 5 similar results instead of just one
  # results = db.similarity_search(query, k=5)

  # # Combine the content of the top 5 results
  # combined_response = "\n\n".join([result.page_content for result in results])

  # # Incorporate the combined context into the query
  # query_with_context = query + "\n\nContext:\n" + combined_response

  # # Generate response using the query with combined context
  # response = generate(query_with_context)

  # stream = client.chat.completions.create(
  #       model="gpt-3.5-turbo",
  #       messages=[{"role": "user", "content": query}],
  #       stream=True,
  #   )
  # for chunk in stream:
  #     if chunk.choices[0].delta.content is not None:
  #         response = response + chunk.choices[0].delta.content
  #response = generate (query)

  return response

Code for Text file

In [ ]:
def text_file(path, message):
  #implement text query logic here

  loader = TextLoader(path)
  text_documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 200)
  documents = text_splitter.split_documents(text_documents)

  db = Chroma.from_documents(documents, OpenAIEmbeddings(openai_api_key = userdata.get('OPENAPIKEY')))

  query = message

  result = db.similarity_search(query)

  response = result[0].page_content

  # query = query + response

  # stream = client.chat.completions.create(
  #       model="gpt-3.5-turbo",
  #       messages=[{"role": "user", "content": query}],
  #       stream=True,
  #   )
  # for chunk in stream:
  #     if chunk.choices[0].delta.content is not None:
  #         response = response + chunk.choices[0].delta.content

  return response

Interface

In [ ]:
########################### GLOBAL VARIABLES ####################################
user_mode = None
path_file = None    #path to the current file uploaded

############################## FUNCTIONS ########################################
def echo(message, history):
    #message holds the input query from the user
    #history holds the chat history in the form of a list containing lists.
    #The inner list structure: [{User text},{Bot text}]

    print(history)

    #No mode
    if(user_mode==None):
      return "Please select an option"

    #web link
    if(user_mode==3):
      url_pattern = re.compile(r'https?://\S+')

      # Search for a URL in the message text
      matchCon = url_pattern.search(message)

      if matchCon:
          url = matchCon.group(0)  # Extract the matched URL
          response = web_link(link=url)
      else:
          #function to give query to appropriate web link function and return response from it
          response = web_link(query=message)

      return response

    #text file mode
    if(user_mode==2):
      return text_file(path_file, message) #path to current .txt file uploaded for this mode

    #pdf file mode
    if(user_mode==1):
      return pdf_file(path_file, message)  #path to current .pdf file uploaded for this mode

#---------------------------------------------------------------------------------------------#

def process_files(argument):

    global path_file
    path_file = argument

    return gr.File(label="Upload File", visible=True,interactive=True)

#---------------------------------------------------------------------------------------------#

def change_mode(choice):
    global user_mode

    if choice == "Web Link":
        user_mode = 3
        return gr.File(visible=False), gr.Button("Submit", visible=False), gr.Button("Cancel", visible=False)
    elif choice == "Text File":
        user_mode = 2
        return gr.File(label="Upload File",visible=True), gr.Button("Submit", visible=True), gr.Button("Cancel", visible=True)
    else:
        user_mode = 1
        return gr.File(label="Upload File",visible=True), gr.Button("Submit", visible=True), gr.Button("Cancel", visible=True)

#--------------------------------------------------------------------------------------------#

def cancel_upload():
    return gr.File(label="Upload File", visible=True,interactive=True)

############################## INTERFACE CODE ###################################
with gr.Blocks() as demo:
    #set up radio element and file input
    with gr.Row():
      radio = gr.Radio(
          ["PDF File", "Text File", "Web Link"], label="Select Mode"
      )

      # input = gr.Interface(process_files,inputs='files',outputs=None)
      with gr.Column():
        file_input = gr.File(label="Upload File", visible=False,interactive=True)

        #set up buttons
        with gr.Row():
            submit_btn = gr.Button("Submit", visible=False)
            cancel_btn = gr.Button("Cancel", visible=False)


    #add radio element event listener
    radio.change(fn=change_mode, inputs=radio, outputs=[file_input,submit_btn,cancel_btn])

    # Set up submit button to process files
    submit_btn.click(fn=process_files, inputs=[file_input], outputs=[file_input])

    # Set up cancel button to clear file input
    cancel_btn.click(fn=cancel_upload, inputs=None, outputs=file_input)

    #set up Chat Interface
    gr.ChatInterface(
        fn=echo,
        title="Doc Bot",
    )

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7a5d9b158bf47c80e8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[]


In [ ]:
def(input):
  //vectorize the input
  outpuit  = model.predict(input)

  return output